In [31]:
from datasets import load_dataset
from transformers import AutoFeatureExtractor  , AutoModelForImageClassification, TrainingArguments, Trainer
from torch.utils.data import Dataset
from torchvision import transforms
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from PIL import Image
import torch

In [36]:
#Load  dataset
dataset = load_dataset("Piro17/dataset-affecthqnet-fer2013")

sample_train = 8000
sample_test = 1000
print(dataset)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

dataset['test'] = dataset['train'].shuffle(seed=96).select(range(sample_test))
dataset['train'] = dataset['train'].shuffle(seed=23).select(range(sample_train))

test_valid_split = dataset['test'].train_test_split(test_size=0.65, seed=45)
dataset['test'] = test_valid_split['train']
dataset['validation'] = test_valid_split['test']

#Import Moel from HuggingFace

model = "google/mobilenet_v2_1.0_224"
#model = "microsoft/resnet-26"
feature_extractor = AutoFeatureExtractor.from_pretrained(model)
model = AutoModelForImageClassification.from_pretrained(model)


DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 56532
    })
})
cuda


In [37]:
print(dataset['train'][0])

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1116x1116 at 0x2E8714BD9D0>, 'label': 5}


In [38]:
data_transforms = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomResizedCrop(size=(48, 48), scale=(0.8, 1.0)),
    transforms.ToTensor()
])

In [39]:
# Define the transform function
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([img.convert("RGB") for img in example_batch['image']], return_tensors='pt')
    inputs['labels'] = example_batch['label']
    return inputs

# Apply the transform to the datasets
dataset = dataset.map(transform, batched=True)

# Remove the 'image' column as it's now transformed
dataset = dataset.remove_columns(['image'])

# Set the format for PyTorch
dataset.set_format(type='torch')

Map: 100%|██████████| 650/650 [00:30<00:00, 21.54 examples/s]


In [40]:
from transformers import Trainer

In [41]:
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [42]:
from transformers import EarlyStoppingCallback

In [ ]:
#Training Args
training_args = TrainingArguments(
    output_dir='./huggingface_fer_model/results',          # output directory
    num_train_epochs=25,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size for training
    per_device_eval_batch_size=32,   # batch size for evaluation
    evaluation_strategy="epoch",     # evaluation strategy to use at the end of each epoch
    save_strategy="epoch",           # save strategy to use at the end of each epoch
    logging_dir='./huggingface_fer_model/logs',            # directory for storing logs
    logging_steps=25,
    warmup_steps=55,                 # number of warmup steps for learning rate scheduler
    report_to=[],                    # disable reporting to any integration
    learning_rate=7e-5,
    weight_decay=0.055,
    fp16=True,                     # use mixed precision training
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    metric_for_best_model="eval_loss",
    greater_is_better=False,          # lower loss is better
    save_total_limit=2,               # limit the total amount of checkpoints, delete the older checkpoints in the output_dir    
)

#Trainer
trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=dataset['train'],      
    eval_dataset=dataset['validation'],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5) ]  # Stop training if no improvement
)

c:\Users\mtlun\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [46]:
trainer.train()


 28%|██▊       | 929/3340 [07:20<17:01,  2.36it/s]

{'loss': 0.6082, 'grad_norm': 10.3058443069458, 'learning_rate': 6.363636363636363e-05, 'epoch': 0.2}



 28%|██▊       | 929/3340 [07:33<17:01,  2.36it/s]

{'loss': 0.6135, 'grad_norm': inf, 'learning_rate': 6.94560327198364e-05, 'epoch': 0.3}



 28%|██▊       | 929/3340 [07:49<17:01,  2.36it/s]

...






















                                                   

 28%|██▊       | 929/3340 [23:39<17:01,  2.36it/s]



{'eval_loss': 1.8702986240386963, 'eval_accuracy': 0.6076923076923076, 'eval_precision': 0.619034545636202, 'eval_recall': 0.6076923076923076, 'eval_f1': 0.6100188655190375, 'eval_runtime': 13.9271, 'eval_samples_per_second': 46.671, 'eval_steps_per_second': 1.508, 'epoch': 6.0}



 60%|██████    | 1500/2500 [16:46<11:11,  1.49it/s]

{'train_runtime': 1006.8732, 'train_samples_per_second': 79.454, 'train_steps_per_second': 2.483, 'train_loss': 0.23589598234494527, 'epoch': 6.0}


TrainOutput(global_step=1500, training_loss=0.23589598234494527, metrics={'train_runtime': 1006.8732, 'train_samples_per_second': 79.454, 'train_steps_per_second': 2.483, 'total_flos': 1.51998969249792e+17, 'train_loss': 0.23589598234494527, 'epoch': 6.0})

In [47]:
eval_results = trainer.evaluate()
print(f"Validation Loss: {eval_results['eval_loss']:.4f}")
print(f"Validation Accuracy: {eval_results['eval_accuracy']:.4f}")
print(f"Validation Precision: {eval_results['eval_precision']:.4f}")
print(f"Validation Recall: {eval_results['eval_recall']:.4f}")
print(f"Validation F1 Score: {eval_results['eval_f1']:.4f}")

100%|██████████| 21/21 [00:08<00:00,  2.59it/s]

Validation Loss: 1.3254
Validation Accuracy: 0.5754
Validation Precision: 0.6255
Validation Recall: 0.5754
Validation F1 Score: 0.5876


In [55]:
model.save_pretrained('./mobilenet_v2_fer2013_model')
feature_extractor.save_pretrained('./mobilenet_v2_fer2013_model')

['./mobilenet_v2_fer2013_model\\preprocessor_config.json']

In [ ]:
model.save_pretrained('./resnet26_fer2013_model')
feature_extractor.save_pretrained('./resnet26_fer2013_model')
